<a href="https://colab.research.google.com/github/agreenbhm/AI-Notebooks/blob/main/Copy_of_Colab_TextGen_Alpaca_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 1. Install the web UI

save_logs_to_google_drive = False #@param {type:"boolean"}
Model = "GPT4-X-Alpaca-13B" #@param ["GPT4-X-Alpaca-13B"]

if save_logs_to_google_drive:
  import os
  import shutil
  from google.colab import drive
  drive.mount('/content/drive')
  base_folder = '/content/drive/MyDrive'

%cd /content
!git clone https://github.com/oobabooga/text-generation-webui
if save_logs_to_google_drive:
  if not os.path.exists(f"{base_folder}/oobabooga-data"):
    os.mkdir(f"{base_folder}/oobabooga-data")
  if not os.path.exists(f"{base_folder}/oobabooga-data/logs"):
    os.mkdir(f"{base_folder}/oobabooga-data/logs")
  if not os.path.exists(f"{base_folder}/oobabooga-data/softprompts"):
    os.mkdir(f"{base_folder}/oobabooga-data/softprompts")
  if not os.path.exists(f"{base_folder}/oobabooga-data/characters"):
    shutil.move("text-generation-webui/characters", f"{base_folder}/oobabooga-data/characters")
  else:
    !rm -r "text-generation-webui/characters"
    
  !rm -r "text-generation-webui/softprompts"
  !ln -s "$base_folder/oobabooga-data/logs" "text-generation-webui/logs"
  !ln -s "$base_folder/oobabooga-data/softprompts" "text-generation-webui/softprompts"
  !ln -s "$base_folder/oobabooga-data/characters" "text-generation-webui/characters"

else:
  !mkdir text-generation-webui/logs

!ln -s text-generation-webui/logs .
!ln -s text-generation-webui/characters .
!ln -s text-generation-webui/models .
%rm -r sample_data
%cd text-generation-webui
!wget https://oobabooga.github.io/settings-colab.json -O settings-colab-template.json

# Install requirements
!pip install -r requirements.txt
!pip install -r extensions/google_translate/requirements.txt
!pip install -r extensions/silero_tts/requirements.txt
print(f"\033[1;32;1m\n --> If you see a warning about \"pydevd_plugins\", just ignore it and move on to Step 3. There is no need to restart the runtime.\n\033[0;37;0m")
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!apt install ./cloudflared-linux-amd64.deb aria2
!rm cloudflared-linux-amd64.deb

if Model == "GPT4-X-Alpaca-13B":
  # Download LLaMA-13B model using aria2c
  !mkdir repositories ; cd repositories ; git clone https://github.com/oobabooga/GPTQ-for-LLaMa.git -b cuda ; cd GPTQ-for-LLaMa ; python setup_cuda.py install
  !export GIT_LFS_SKIP_SMUDGE=1 ; cd models ; git clone https://huggingface.co/anon8231489123/gpt4-x-alpaca-13b-native-4bit-128g ; cd gpt4-x-alpaca-13b-native-4bit-128g; rm gpt-x-alpaca-13b-native-4bit-128g-cuda.pt gpt-x-alpaca-13b-native-4bit-128g.pt training_args.bin tokenizer.model gpt4-x-alpaca-13b-ggml-q4_1-from-gptq-4bit-128g/ggml-model-q4_1.bin ; wget https://huggingface.co/anon8231489123/gpt4-x-alpaca-13b-native-4bit-128g/resolve/main/training_args.bin ; wget https://huggingface.co/anon8231489123/gpt4-x-alpaca-13b-native-4bit-128g/resolve/main/tokenizer.model ; aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/anon8231489123/gpt4-x-alpaca-13b-native-4bit-128g/resolve/main/gpt-x-alpaca-13b-native-4bit-128g-cuda.pt -o gpt-x-alpaca-13b-native-4bit-128g-cuda.pt


In [ ]:
#@title 2. Launch

!pkill cloudflared
!rm nohup.out 2> /dev/null
!nohup cloudflared --url http://127.0.0.1:7860 &
!sleep 5 && grep --color=never -o 'https://.*trycloudflare.com' < nohup.out | sed 's/^/Once the local URL below shows, the Cloudflare URL is: /'

if Model == "GPT4-X-Alpaca-13B":
  # Launch the model with specific parameters
  !python server.py --model gpt4-x-alpaca-13b-native-4bit-128g --wbits 4 --groupsize 128